# LP Model

In [1]:
import numpy as np
import mcf_simplex_analyzer.fractionarray as fa
import mcf_simplex_analyzer.fractionarray.sparse as fas
from mcf_simplex_analyzer.simplex import LPModel, InequalityType

In [2]:
model = LPModel(name="test_problem")
model

LPModel(name='test_problem', constraints=[], objective_function=None, variable_no=0)

In [3]:
xs = [ model.new_variable(f"x_{i}") for i in range(4) ]
xs

[Variable(index=0, name='x_0'),
 Variable(index=1, name='x_1'),
 Variable(index=2, name='x_2'),
 Variable(index=3, name='x_3')]

In [4]:
c1 = 3 * xs[0] + 2 * xs[1] + xs[2] + 2 * xs[3] <= 225
c2 = xs[0] + xs[1] + xs[3] + xs[3] <= 117 
c3 = 4 * xs[0] + 3 * xs[1] + 3 * xs[2] + 4 * xs[3] <= 420
model.constraints.extend((c1, c2, c3))

print(c1, c2, c3, sep='\n')

Constraint(combination=LinearCombination(combination=[FactorVar(factor=3, var=Variable(index=0, name='x_0')), FactorVar(factor=2, var=Variable(index=1, name='x_1')), FactorVar(factor=1, var=Variable(index=2, name='x_2')), FactorVar(factor=2, var=Variable(index=3, name='x_3'))]), right_hand_side=225, type=<InequalityType.LE: 0>)
Constraint(combination=LinearCombination(combination=[FactorVar(factor=1, var=Variable(index=1, name='x_1')), FactorVar(factor=1, var=Variable(index=0, name='x_0')), FactorVar(factor=2, var=Variable(index=3, name='x_3'))]), right_hand_side=117, type=<InequalityType.LE: 0>)
Constraint(combination=LinearCombination(combination=[FactorVar(factor=4, var=Variable(index=0, name='x_0')), FactorVar(factor=3, var=Variable(index=1, name='x_1')), FactorVar(factor=3, var=Variable(index=2, name='x_2')), FactorVar(factor=4, var=Variable(index=3, name='x_3'))]), right_hand_side=420, type=<InequalityType.LE: 0>)


In [5]:
objective_function = 19 * xs[0] + 13 * xs[1] + 12 * xs[2] + 17 * xs[3]
model.objective_function = objective_function
model.objective_function

LinearCombination(combination=[FactorVar(factor=19, var=Variable(index=0, name='x_0')), FactorVar(factor=13, var=Variable(index=1, name='x_1')), FactorVar(factor=12, var=Variable(index=2, name='x_2')), FactorVar(factor=17, var=Variable(index=3, name='x_3'))])

In [6]:
shape = (len(model.constraints), model.variable_no)

cols = []
rows = []
data = []
rhs = []
for row, constraint in enumerate(model.constraints):
    if constraint.type == InequalityType.GE:
        constraint = -constraint

    rhs.append(constraint.right_hand_side)
    for fv in constraint.combination:
        cols.append(fv.var.index)
        rows.append(row)
        data.append(fv.factor)

c = fa.zeros(model.variable_no)
for fv in objective_function.combination:
    c[fv.var.index] = fv.factor

tmp = fas.FractionCOOMatrix(
    shape=shape,
    col=np.array(cols, dtype=np.int64),
    row=np.array(rows, dtype=np.int64),
    data=fa.FractionArray.from_array(data),
)
table = fas.coo_to_csc(tmp)
b = fa.FractionArray.from_array(rhs)
table, b, c

(FractionCSCMatrix(shape=(3, 4), indptr=array([ 0,  3,  6,  8, 11]), indices=array([0, 1, 2, 0, 1, 2, 0, 2, 0, 1, 2]), data=FractionArray(numerator=[3 1 4 2 1 3 1 3 2 2 4], denominator=[1 1 1 1 1 1 1 1 1 1 1])),
 FractionArray(numerator=[225 117 420], denominator=[1 1 1]),
 FractionArray(numerator=[19 13 12 17], denominator=[1 1 1 1]))